In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
data = pd.read_csv('health_dataset.csv')

# Defining dictionaries to map medications and illnesses to seriousness levels
medication_seriousness = {
    "Chemotherapy drugs": 3,
    "Immunosuppressants": 3,
    "Anticoagulants": 3,
    "Insulin": 3,
    "Opioids": 3,
    "Antibiotics": 2,
    "Antihypertensives": 2,
    "Statins": 2,
    "Antidepressants": 2,
    "Antihistamines": 2,
    "Over-the-counter pain relievers": 1,
    "Cough and cold medications": 1,
    "Allergy medications": 1,
    "Digestive aids": 1,
    "Vitamins and supplements": 1
}

illness_seriousness = {
    "Stroke": 3,
    "Heart disease": 3,
    "Diabetes": 3,
    "Chronic obstructive pulmonary disease (COPD)": 3,
    "Cancer": 3,
    "Arthritis": 2,
    "Asthma": 2,
    "Alzheimer's disease": 2,
    "Depression": 2,
    "Kidney disease": 2,
    "Common cold": 1,
    "Flu": 1,
    "Bronchitis": 1,
    "Sinusitis": 1,
    "Ear infection": 1
}

# Mapping the "Medication" and "Existing_illness" columns to seriousness levels
data["Medication_Seriousness"] = data["Medication"].map(medication_seriousness)
data["Existing_Illness_Seriousness"] = data["Existing_illness"].map(illness_seriousness)
data["Previous_Illness_Seriousness"] = data["Previous_illness"].map(illness_seriousness)

# Mapping the "Location" column to seriousness levels
location_val = {
    "Metro City": 2,
    "City": 1,
    "Town": 0
}
data["Location_Value"] = data["Location"].map(location_val)

# Selecting features and target
X = data[["Age", "Medication_Seriousness", "Existing_Illness_Seriousness", "Previous_Illness_Seriousness", "Location_Value"]]
y = data["Score"]

# Standardizing numeric features
scaler = StandardScaler()
X_copy = X.copy()  # Make a copy of the DataFrame
X_copy[["Age"]] = scaler.fit_transform(np.array(X_copy["Age"]).reshape(-1, 1))
X = X_copy

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# To ensure the correct data types
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')

# Defining the input shape for the model
input_shape = (X_train.shape[1],)

# Building a feedforward neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape, name="input_layer"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1, name="output_layer")  # Output layer for regression
])

# Compiling the model
model.compile(optimizer="adam", loss="mean_squared_error")

# Training the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=2)

# Evaluating the model
mse = model.evaluate(X_test, y_test)
print("Mean Squared Error:", mse)


Epoch 1/100
100/100 - 1s - loss: 0.7990 - val_loss: 0.2109 - 848ms/epoch - 8ms/step
Epoch 2/100
100/100 - 0s - loss: 0.1846 - val_loss: 0.1710 - 157ms/epoch - 2ms/step
Epoch 3/100
100/100 - 0s - loss: 0.1605 - val_loss: 0.1552 - 168ms/epoch - 2ms/step
Epoch 4/100
100/100 - 0s - loss: 0.1486 - val_loss: 0.1451 - 163ms/epoch - 2ms/step
Epoch 5/100
100/100 - 0s - loss: 0.1442 - val_loss: 0.1414 - 152ms/epoch - 2ms/step
Epoch 6/100
100/100 - 0s - loss: 0.1353 - val_loss: 0.1307 - 160ms/epoch - 2ms/step
Epoch 7/100
100/100 - 0s - loss: 0.1266 - val_loss: 0.1215 - 158ms/epoch - 2ms/step
Epoch 8/100
100/100 - 0s - loss: 0.1159 - val_loss: 0.1221 - 165ms/epoch - 2ms/step
Epoch 9/100
100/100 - 0s - loss: 0.1097 - val_loss: 0.1103 - 160ms/epoch - 2ms/step
Epoch 10/100
100/100 - 0s - loss: 0.1046 - val_loss: 0.0955 - 161ms/epoch - 2ms/step
Epoch 11/100
100/100 - 0s - loss: 0.0985 - val_loss: 0.0968 - 160ms/epoch - 2ms/step
Epoch 12/100
100/100 - 0s - loss: 0.0944 - val_loss: 0.0883 - 160ms/epoch 

Epoch 98/100
100/100 - 0s - loss: 0.0754 - val_loss: 0.0776 - 177ms/epoch - 2ms/step
Epoch 99/100
100/100 - 0s - loss: 0.0754 - val_loss: 0.0773 - 163ms/epoch - 2ms/step
Epoch 100/100
100/100 - 0s - loss: 0.0782 - val_loss: 0.0787 - 165ms/epoch - 2ms/step
32/32 [==============================] - 0s 1ms/step - loss: 0.1065
Mean Squared Error: 0.10650643706321716


In [2]:
# Saving the trained model to a file
model.save("trained_model.h5")